# Mustererkennung/Machine Learning - Assignment 8



In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [14]:
class Classifier:
    
    def accuracy(self, labels, predictions):
        return np.mean(labels == predictions)


data = pd.read_csv("./Data/iris.data", header=None)
data.head(n=5)

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


#### Splitting the data into training/test and according to their class memberships

In [37]:
X_train, X_test, y_train, y_test = train_test_split(data[list(range(4))], data[4], test_size=0.2, random_state=None, stratify=data[4])

X_train_setosa = X_train[y_train=='Iris-setosa'].to_numpy()
X_train_versicolor = X_train[y_train=='Iris-versicolor'].to_numpy()
X_train_virginica = X_train[y_train=='Iris-virginica'].to_numpy()

y_train_setosa = y_train[y_train=='Iris-setosa'].to_numpy()
y_train_versicolor = y_train[y_train=='Iris-versicolor'].to_numpy()
y_train_virginica = y_train[y_train=='Iris-virginica'].to_numpy()

X_test_setosa_v_v = X_test.to_numpy()
y_test_setosa_v_v = (y_test == 'Iris-setosa').astype(int).to_numpy()

X_test_versicolor_virginica = X_test[y_test!='Iris-setosa'].to_numpy()
y_test_versicolor_virginica = (y_test[y_test!='Iris-setosa'] == 'Iris-versicolor').astype(int).to_numpy()

In [16]:
X_train.shape

(120, 4)

In [27]:
X_train_setosa.shape

(40, 4)

In [29]:
X_test_setosa_v_v.shape

(30, 4)

In [33]:
y_test_setosa_v_v

array([1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 1])

In [32]:
y_test_versicolor_virginica.shape

(20,)

In [20]:
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

## Task 1:  seperate Setosa from Versicolor and Virginica 

In [52]:
def sub_norm(w_1, w_2):
    return np.linalg.norm(w_1-w_2)

In [259]:
class Perceptron:
    def __init__(self, w, theta):
        self.w = w
        self.theta = theta
    
    def fit(self, X_pos, X_neg, y_pos, y_neg):
        # initialize the w
        w_1 = np.mean(X_pos, 0)
        
        X_data = np.concatenate((X_pos, X_neg), axis=0)
        
        n_data = X_data.shape[0]
        n_pos = X_pos.shape[0]
        
        for i in range(10000): 
            
            w = w_1
            
            index = np.random.randint(n_data)
            v = X_data[index]
            
            if(index < n_pos and w.T @ v > 0):
                continue
            
            if(index < n_pos and w.T @ v <= 0):
                w_1 = w + v
            
            if(index >= n_pos and w.T @ v < 0):
                continue
            
            if(index >= n_pos and w.T @ v >= 0):
                w_1 = w - v
            
            if(sub_norm(w_1, w) <= self.theta):
                break
        self.w = w_1
        
    def calculate(self, v):
        if(self.w.T @ v > 0):
            return 1
        else:
            return 0
        
    def predict(self, X_data):
        num = X_data.shape[0]
        res = np.zeros(num)
        
        for i in range(num):
            if(self.calculate(X_data[i]) == 1):
                res[i] = 1
                
        return res
        
    def accuracy(self, labels, predictions):
        return np.mean(labels == predictions)
        

In [260]:
perceptron = Perceptron(np.mean(X_train_setosa, 0),1e-3)

In [261]:
perceptron.fit(X_train_setosa, np.concatenate((X_train_versicolor, X_train_virginica)), y_train_setosa, np.concatenate((y_train_versicolor, y_train_virginica)))
predictions = perceptron.predict(X_test_setosa_v_v)
print(perceptron.accuracy(y_test_setosa_v_v, predictions))

0.3333333333333333


In [204]:
def sub_vec(X_data, v):
    for i in range(len(X_data)):
        X_data[i] -= v
    return X_data

In [263]:
X_test_versicolor = X_test[y_test=='Iris-versicolor']
y_test_versicolor = y_test[y_test=='Iris-versicolor']

temp = np.mean(X_train, 0)

perceptron.fit(X_train_versicolor, X_train_virginica,y_train_versicolor, y_train_virginica)
predictions = perceptron.predict(X_test_versicolor_virginica)
print(perceptron.accuracy(y_test_versicolor_virginica, predictions))

<ipython-input-263-73853fa98371>:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  X_test_versicolor = X_test[y_test=='Iris-versicolor']
<ipython-input-263-73853fa98371>:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  y_test_versicolor = y_test[y_test=='Iris-versicolor']


0.5


__The problem while classifying Versicolor from Virginica__ is that all vector have positive projection on w so that with perceptron we can't get right separation. 

# Task 2 MLP

In [264]:
from sklearn.linear_model import Perceptron

In [265]:
path_to_train = './Data/zip.train'
path_to_test = './Data/zip.test'
training_data = np.array(pd.read_csv(path_to_train, sep=' ', header=None))
test_data = np.array(pd.read_csv(path_to_test, sep =' ',header=None))

X_train, y_train = training_data[:,1:-1], training_data[:,0]
X_test, y_test = test_data[:,1:], test_data[:,0]

In [268]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
# print(X_train[0])

(7291, 256)
(2007, 256)
(7291,)
(2007,)


In [281]:
class ANN:
    '''
    depth's shape: d(constant)   n_neuron's shape 1*d
    '''
    def __init__(self, depth, n_neuron):
        self.depth = depth
        self.n_neuron = n_neuron
        
        self.input = []
        self.weight = []
        self.b = []
        
    def init_value(self, X_data):
        # 256 * 1
        before = X_data.shape[0]
        for i in self.n_neuron:
            # i * 256 --> i * 1
            temp_weight = np.random.random((i, before))
            temp_b = np.random.random((i, 1))
            
            self.weight.append(temp_weight)
            self.b.append(temp_b)
            
            before = i
            
    def sigmod(self, x):
        return 1 / (1 + np.exp(-x))
            
    def heaven_function(self, x):
        return self.sigmod(x)
        
    def forward_prop(self, X_data):
        self.init_value(X_data.T)
        
        self.input = X_data.T
        
        # go through all depths
        for d in range(self.depth):
            temp = self.weight[d] @ self.input + self.b[d]
            self.input = self.heaven_function(temp)
            
        return self.input

In [283]:
ann = ANN(2, [2, 3])

for x in X_train:
    print(ann.forward_prop(x))

[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402409]
 [0.70547034 0.70547041]
 [0.68591477 0.68591484]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403

[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.57789805 0.6895289 ]
 [0.7837575  0.87654339]
 [0.77476975 0.87799333]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403

 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477

 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477

 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402407 0.50402407]
 [0.70547039 0.70547038]
 [0.68591482 0.68591481]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477

...

[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.5040257  0.50402544]
 [0.70547227 0.70547197]
 [0.68591696 0.68591662]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403

[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403

 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477

[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.69673034 0.69673038]
 [0.88154689 0.88154692]
 [0.88342323 0.88342326]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403

 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.51436678 0.50577876]
 [0.71727779 0.70749315]
 [0.6993406  0.6882147 ]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.54280142 0.60595457]
 [0.74831072 0.80987235]
 [0.7346099  0.80418275]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477 0.68591477]]
[[0.50402403 0.50402403]
 [0.70547034 0.70547034]
 [0.68591477